In [ ]:
# Parámetros + utilidades

library(ape)
library(readr)

# Árbol MCC
TREE_MCC_FILE  <- "../trees/BEAST_MCC_46_ultrametric_FIXED.tre"

# Carpeta 10 sets (desde Python)
SETS_DIR <- "../traits/gradient_sets"

# Patrón esperado de archivos
SET_PATTERN <- "^traits_high-level_set[0-9]{2}\\.csv$"

# Salida: sets alineados al MCC (mismo directorio)
ALIGNED_SUFFIX <- "_alignedMCC"  # quedarán como traits_high-level_set01_alignedMCC.csv

# Lectura del MCC
tree <- read.tree(TREE_MCC_FILE)
stopifnot(inherits(tree, "phylo"))

cat("OK — MCC cargado\n")
cat("Tips en MCC:", length(tree$tip.label), "\n")
cat("Ejemplo tip labels:", paste(head(tree$tip.label, 5), collapse = " | "), "\n\n")

# normalizar nombres de especie
norm_sp <- function(x) {
  x <- trimws(x)
  x <- gsub(" ", "_", x)
  x
}


# Listar archivos de sets
stopifnot(dir.exists(SETS_DIR))
set_files <- list.files(SETS_DIR, pattern = SET_PATTERN, full.names = TRUE)
set_files <- sort(set_files)

cat("Sets encontrados:", length(set_files), "\n")
print(basename(set_files))

if (length(set_files) != 10) {
  warning("No encontré exactamente 10 sets con el patrón ", SET_PATTERN,
          "\nRevisa nombres en: ", normalizePath(SETS_DIR))
}

cat("\nOK \n")


In [ ]:
# Alinear sets al MCC y guardar CSVs

aligned_files <- character(0)

for (f in set_files) {

  df <- read_csv(f, show_col_types = FALSE)

  # Espera : primera columna = species (desde script Python)
  if (!("species" %in% names(df))) {
    stop("El archivo no tiene columna 'species': ", basename(f))
  }

  # Pasar a rownames y limpiar
  sp_raw <- df$species
  sp <- norm_sp(sp_raw)

  df$species <- NULL
  mat <- as.data.frame(df)
  rownames(mat) <- sp

  # Chequeos contra MCC
  tips <- norm_sp(tree$tip.label)

  # ¿Coinciden los conjuntos?
  if (!setequal(rownames(mat), tips)) {

    missing_in_set <- setdiff(tips, rownames(mat))
    extra_in_set   <- setdiff(rownames(mat), tips)

    cat("\nERROR en:", basename(f), "\n")
    cat("  Faltan en set (presentes en MCC):", length(missing_in_set), "\n")
    if (length(missing_in_set) > 0) cat("   ", paste(missing_in_set, collapse = " | "), "\n")

    cat("  Sobran en set (no están en MCC):", length(extra_in_set), "\n")
    if (length(extra_in_set) > 0) cat("   ", paste(extra_in_set, collapse = " | "), "\n")

    stop("No hay match 1:1 entre tips del MCC y filas del set: ", basename(f))
  }

  # Reordenar como MCC
  mat2 <- mat[tips, , drop = FALSE]

  # Guardar
  out_name <- sub("\\.csv$", paste0(ALIGNED_SUFFIX, ".csv"), basename(f))
  out_path <- file.path(SETS_DIR, out_name)

  out_df <- data.frame(species = rownames(mat2), mat2, check.names = FALSE)
  write_csv(out_df, out_path)

  aligned_files <- c(aligned_files, out_path)

  cat("alineado:", basename(f), "->", basename(out_path),
      "| dim:", nrow(out_df), "x", ncol(out_df), "\n")
}

cat("\nArchivos alineados generados:", length(aligned_files), "\n")
print(basename(aligned_files))

cat("\nOK \n")


In [ ]:

# Parámetros ASR + función para correr 1 set


library(ape)
library(readr)
library(mvMORPH)

#  Salida
OUT_DIR <- "../results"
OUT_SUBDIR <- file.path(OUT_DIR, "asr_MCC_OU_gradient_sets")
dir.create(OUT_SUBDIR, recursive = TRUE, showWarnings = FALSE)

#  Configuración del modelo
MODEL   <- "OU"
PENALTY <- "RidgeArch"
METHOD  <- "H&L"

# Diagnóstico
DROP_SD_ZERO   <- TRUE
DO_COR_SUMMARY <- TRUE
COR_SAMPLE_MAX <- 5e5
SEED_COR       <- 123

# Asegurar que existen los objetos previos
stopifnot(exists("tree"), inherits(tree, "phylo"))
stopifnot(exists("aligned_files"))
stopifnot(length(aligned_files) >= 1)

# Helper: correr ASR completo para 1 set y exportar nodos
run_asr_one_set <- function(traits_csv, tree, out_csv,
                            drop_sd_zero = TRUE,
                            do_cor_summary = TRUE,
                            cor_sample_max = 5e5,
                            seed_cor = 123) {

  # Leer traits
  traits_tbl <- readr::read_csv(traits_csv, show_col_types = FALSE)
  traits_df <- as.data.frame(traits_tbl)

  if (!("species" %in% names(traits_df))) {
    stop("No existe columna 'species' en: ", basename(traits_csv))
  }

  rownames(traits_df) <- norm_sp(traits_df$species)
  traits_df$species   <- NULL

  # Asegurar numérico
  traits_df[] <- lapply(traits_df, function(z) suppressWarnings(as.numeric(z)))

  # Alinear con MCC
  tips <- norm_sp(tree$tip.label)

  if (!setequal(rownames(traits_df), tips)) {
    miss <- setdiff(tips, rownames(traits_df))
    extra <- setdiff(rownames(traits_df), tips)
    stop(
      "Mismatch árbol ↔ rasgos en ", basename(traits_csv),
      "\n  faltan: ", paste(miss, collapse=" | "),
      "\n  sobran: ", paste(extra, collapse=" | ")
    )
  }

  traits_df <- traits_df[tips, , drop = FALSE]

  # Diagnóstico NA
  na_total <- sum(is.na(traits_df))
  if (na_total > 0) {
    stop("Hay NA en la matriz de rasgos (", na_total, ") en: ", basename(traits_csv))
  }

  # Quitar sd==0
  sds     <- apply(traits_df, 2, sd)
  idx_sd0 <- which(!is.na(sds) & sds == 0)

  traits_proc <- traits_df
  if (drop_sd_zero && length(idx_sd0) > 0) {
    traits_proc <- traits_df[, -idx_sd0, drop = FALSE]
  }

  # Resumen correlaciones
  cor_quantiles <- NULL
  if (do_cor_summary && ncol(traits_proc) >= 2) {
    set.seed(seed_cor)
    C   <- suppressWarnings(cor(traits_proc))
    ut  <- upper.tri(C, diag = FALSE)
    cors <- C[ut]
    rm(C, ut)

    if (length(cors) > cor_sample_max) {
      cors <- sample(cors, cor_sample_max)
    }

    cor_quantiles <- quantile(cors, probs = c(0, .05, .25, .5, .75, .95, 1), na.rm = TRUE)
  }

  # Ajuste OU penalizado (mvgls)
  Y <- as.matrix(traits_proc)
  storage.mode(Y) <- "double"
  dat <- list(Y = Y)

  t0 <- Sys.time()
  fit_ou <- mvgls(
    Y ~ 1,
    data    = dat,
    tree    = tree,
    model   = MODEL,
    penalty = PENALTY,
    method  = METHOD
  )
  t1 <- Sys.time()
  fit_sec <- as.numeric(difftime(t1, t0, units = "secs"))

  # ASR nodal
  t2 <- Sys.time()
  A_nodes <- mvMORPH::ancestral(fit_ou)  # [Nnode × p]
  t3 <- Sys.time()
  asr_sec <- as.numeric(difftime(t3, t2, units = "secs"))

  Ntip  <- length(tree$tip.label)
  Nnode <- tree$Nnode
  stopifnot(is.matrix(A_nodes), nrow(A_nodes) == Nnode)

  node_ids <- (Ntip + 1):(Ntip + Nnode)

  asr_tbl <- as.data.frame(A_nodes)
  colnames(asr_tbl) <- colnames(traits_proc)
  asr_tbl <- cbind(node = node_ids, asr_tbl)

  readr::write_csv(asr_tbl, out_csv)

  # Resumen para logging
  gic <- GIC(fit_ou)
  ic_value <- if (is.list(gic)) as.numeric(gic$GIC) else as.numeric(gic)[1]
  loglik   <- if (is.list(gic)) as.numeric(gic$LogLikelihood) else NA_real_
  ridge_tuning <- fit_ou$tuning[1]

  msg_cor <- if (!is.null(cor_quantiles)) {
    paste0(
      "Cor qtls: ",
      paste(names(cor_quantiles), sprintf("%.3f", as.numeric(cor_quantiles)), collapse=" | ")
    )
  } else "Cor qtls: (skip)"

  cat(sprintf(
    paste0(
      "OK — ASR 1 set\n",
      "Archivo traits: %s\n",
      "Especies: %d | Rasgos: %d (sd0 removidos: %d)\n",
      "GIC=%.3f | LogLik=%.3f | ridge_tuning=%.6f\n",
      "Fit: %.1f s | ASR: %.1f s\n",
      "%s\n",
      "Guardado nodos: %s\n\n"
    ),
    basename(traits_csv),
    nrow(traits_df), ncol(traits_proc), length(idx_sd0),
    ic_value, loglik, ridge_tuning,
    fit_sec, asr_sec,
    msg_cor,
    out_csv
  ))

  return(list(
    traits_csv = traits_csv,
    out_csv    = out_csv,
    n_species  = nrow(traits_df),
    p_traits   = ncol(traits_proc),
    sd0_removed = length(idx_sd0),
    gic = ic_value,
    loglik = loglik,
    ridge_tuning = ridge_tuning,
    fit_sec = fit_sec,
    asr_sec = asr_sec
  ))
}

cat("OK (función lista). OUT_SUBDIR:", normalizePath(OUT_SUBDIR, mustWork = FALSE), "\n")


In [ ]:

# Loop ASR (10 sets) + outputs

# Control simple: si TRUE, se detiene al primer error ( debug inmediato)
FAIL_FAST <- TRUE

results_list <- list()

`%||%` <- function(a, b) if (!is.null(a)) a else b

for (traits_csv in aligned_files) {

  # Extraer ID del set desde el nombre (set01 ... set10)
  bn <- basename(traits_csv)
  set_id <- sub(".*set([0-9]{2}).*", "\\1", bn)

  out_csv <- file.path(
    OUT_SUBDIR,
    paste0("ASR_MCC_OU_gradient_set", set_id, "_nodes.csv")
  )

  if (FAIL_FAST) {
    res <- run_asr_one_set(
      traits_csv = traits_csv,
      tree       = tree,
      out_csv    = out_csv,
      drop_sd_zero    = DROP_SD_ZERO,
      do_cor_summary  = DO_COR_SUMMARY,
      cor_sample_max  = COR_SAMPLE_MAX,
      seed_cor        = SEED_COR
    )
    results_list[[paste0("set", set_id)]] <- res

  } else {
    res <- tryCatch(
      run_asr_one_set(
        traits_csv = traits_csv,
        tree       = tree,
        out_csv    = out_csv,
        drop_sd_zero    = DROP_SD_ZERO,
        do_cor_summary  = DO_COR_SUMMARY,
        cor_sample_max  = COR_SAMPLE_MAX,
        seed_cor        = SEED_COR
      ),
      error = function(e) {
        cat("ERROR — Set", set_id, ":", conditionMessage(e), "\n\n")
        return(list(
          traits_csv = traits_csv,
          out_csv = out_csv,
          error = conditionMessage(e)
        ))
      }
    )
    results_list[[paste0("set", set_id)]] <- res
  }
}

# Resumen a CSV
summary_tbl <- do.call(rbind, lapply(names(results_list), function(nm) {
  x <- results_list[[nm]]
  data.frame(
    set = nm,
    traits_csv = x$traits_csv %||% NA_character_,
    out_csv    = x$out_csv    %||% NA_character_,
    n_species  = x$n_species  %||% NA_integer_,
    p_traits   = x$p_traits   %||% NA_integer_,
    sd0_removed = x$sd0_removed %||% NA_integer_,
    gic = x$gic %||% NA_real_,
    loglik = x$loglik %||% NA_real_,
    ridge_tuning = x$ridge_tuning %||% NA_real_,
    fit_sec = x$fit_sec %||% NA_real_,
    asr_sec = x$asr_sec %||% NA_real_,
    error = x$error %||% NA_character_,
    stringsAsFactors = FALSE
  )
}))

summary_csv <- file.path(OUT_SUBDIR, "ASR_MCC_OU_gradient_sets_summary.csv")
write_csv(summary_tbl, summary_csv)

cat("OK — Loop terminado.\n")
cat("Resumen:", summary_csv, "\n")
cat("Ejemplos outputs nodales:\n")
print(head(summary_tbl$out_csv, 3))
cat("\nOK \n")


In [ ]:

# Setup: ASR oficial (mediana) + 10 ASR gradient + edades nodales


library(readr)
library(dplyr)
library(ggplot2)

OUT_DIR <- "../results"

# --- ASR oficial (tips = mediana) ---
ASR_REF_FILE <- file.path(OUT_DIR, "asr_MCC_OU_nodes_high-level.csv")

# --- Directorio con  10 ASR nodales (gradient sets) ---
GRAD_DIR     <- file.path(OUT_DIR, "asr_MCC_OU_gradient_sets")
GRAD_PATTERN <- "^ASR_MCC_OU_gradient_set[0-9]{2}_nodes\\.csv$"

# --- Edades nodales (igual que en figura bootstrap) ---
AGE_FILE <- file.path(OUT_DIR, "asr_MCC_OU_bootstrap", "node_uncertainty_with_age.csv")

# --- Salidas (las generaremos en celdas siguientes) ---
OUT_DIST_CSV <- file.path(GRAD_DIR, "gradient_distances_all_nodes.csv")
OUT_SUM_CSV  <- file.path(GRAD_DIR, "gradient_distance_summary_with_age.csv")
OUT_FIG      <- file.path(GRAD_DIR, "asr_sampling_uncertainty_distance_vs_age.png")

# Cargar ASR oficial (referencia)
asr_ref <- readr::read_csv(ASR_REF_FILE, show_col_types = FALSE)

trait_cols <- setdiff(colnames(asr_ref), "node")
if (length(trait_cols) == 0L) {
  stop("No se identificaron columnas de rasgos en ASR_REF_FILE (además de 'node').")
}

cat("ASR REF cargado:", basename(ASR_REF_FILE), "\n")
cat("N nodos:", nrow(asr_ref), " | P rasgos:", length(trait_cols), "\n\n")


# Listar los 10 ASR gradient

stopifnot(dir.exists(GRAD_DIR))
grad_files <- sort(list.files(GRAD_DIR, pattern = GRAD_PATTERN, full.names = TRUE))

cat("Gradient ASR files encontrados:", length(grad_files), "\n")
print(basename(grad_files))

if (length(grad_files) != 10) {
  warning("Esperaba 10 archivos y encontré ", length(grad_files),
          ". Revisa GRAD_PATTERN o el contenido de: ", GRAD_DIR)
}


# Cargar edades nodales

node_age <- readr::read_csv(AGE_FILE, show_col_types = FALSE) %>%
  select(node, age_Mya)

cat("\nEdad nodal cargada:", basename(AGE_FILE), "\n")
cat("N filas edad:", nrow(node_age), "\n")

cat("\nOK — Celda 04\n")


In [ ]:

# Distancias (gradient sets vs ASR oficial) para todos los nodos

# Chequeos 
stopifnot(exists("asr_ref"), exists("trait_cols"), exists("grad_files"))
stopifnot(nrow(asr_ref) > 0, length(trait_cols) > 0, length(grad_files) > 0)

# Mantener ASR ref indexado por node para alinear fácil
asr_ref2 <- asr_ref %>% arrange(node)

# Helper para extraer el ID del set (01..10)
get_set_id <- function(path) {
  bn <- basename(path)
  sub(".*set([0-9]{2}).*", "\\1", bn)
}

dist_all <- list()

for (f in grad_files) {

  set_id <- get_set_id(f)

  # Leer ASR del set
  asr_set <- readr::read_csv(f, show_col_types = FALSE) %>% arrange(node)

  # Chequear columnas
  cols_set <- setdiff(colnames(asr_set), "node")

  if (!setequal(cols_set, trait_cols)) {
    miss <- setdiff(trait_cols, cols_set)
    extra <- setdiff(cols_set, trait_cols)

    cat("\nERROR columnas en:", basename(f), "\n")
    cat("  Faltan (vs ASR oficial):", length(miss), "\n")
    if (length(miss) > 0) cat("   ", paste(head(miss, 20), collapse=" | "), ifelse(length(miss)>20," | ...",""), "\n")
    cat("  Sobran (no están en ASR oficial):", length(extra), "\n")
    if (length(extra) > 0) cat("   ", paste(head(extra, 20), collapse=" | "), ifelse(length(extra)>20," | ...",""), "\n")

    stop("Columnas de rasgos no coinciden entre ASR oficial y ", basename(f))
  }

  # Alinear por node
  if (!setequal(asr_set$node, asr_ref2$node)) {
    miss_nodes <- setdiff(asr_ref2$node, asr_set$node)
    extra_nodes <- setdiff(asr_set$node, asr_ref2$node)
    stop(
      "Mismatch de nodos en ", basename(f),
      "\n  faltan nodos: ", paste(head(miss_nodes, 20), collapse=" | "),
      "\n  sobran nodos: ", paste(head(extra_nodes, 20), collapse=" | ")
    )
  }

  idx <- match(asr_ref2$node, asr_set$node)
  asr_set2 <- asr_set[idx, ]

  # Matrices numéricas en mismo orden de columnas
  M_ref <- as.matrix(asr_ref2[, trait_cols])
  M_set <- as.matrix(asr_set2[, trait_cols])

  storage.mode(M_ref) <- "double"
  storage.mode(M_set) <- "double"

  # Distancia euclidiana por nodo (vectorizada)
  d <- sqrt(rowSums((M_set - M_ref)^2))

  dist_all[[paste0("set", set_id)]] <- data.frame(
    node = asr_ref2$node,
    set  = paste0("set", set_id),
    dist = d
  )

  cat("OK — distancias:", basename(f), "| set", set_id, "| N nodos:", length(d), "\n")
}

dist_tbl <- bind_rows(dist_all)

# Guardar distancias largas
readr::write_csv(dist_tbl, OUT_DIST_CSV)
cat("\nCSV guardado:", OUT_DIST_CSV, "\n")
cat("OK\n")


In [ ]:

# Plot + guarda PNG

library(readr)
library(dplyr)
library(ggplot2)

# Cargar distancias (long) y edades
dist_tbl <- read_csv(OUT_DIST_CSV, show_col_types = FALSE)
node_age <- read_csv(AGE_FILE, show_col_types = FALSE) %>% select(node, age_Mya)

# Resumen por nodo (IC95 empírico sobre 10 sets)
dist_sum <- dist_tbl %>%
  group_by(node) %>%
  summarise(
    dist_median = median(dist, na.rm = TRUE),
    dist_q025   = quantile(dist, 0.025, na.rm = TRUE),
    dist_q975   = quantile(dist, 0.975, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  inner_join(node_age, by = "node")

# Estética
STAR_SIZE    <- 4.0
STAR_LABEL   <- "\u2605"  # ★
LINE_SIZE    <- 1.0
RIBBON_ALPHA <- 0.2
AXIS_TITLE_SIZE <- 13
AXIS_TEXT_SIZE  <- 11

p_dist <- ggplot(dist_sum, aes(x = age_Mya, y = dist_median)) +
  geom_ribbon(aes(ymin = dist_q025, ymax = dist_q975), alpha = RIBBON_ALPHA) +
  geom_line(size = LINE_SIZE) +
  geom_text(label = STAR_LABEL, size = STAR_SIZE, color = "black") +
  scale_x_reverse(
    limits = c(38, 1),
    breaks = seq(1, 38, by = 2),
    expand = expansion(mult = c(0, 0))
  ) +
  scale_y_continuous(expand = expansion(mult = c(0, 0))) +
  labs(
    x = "Node age [Mya]",
    y = "Median Euclidean distance"
  ) +
  theme_bw() +
  theme(
    axis.title = element_text(size = AXIS_TITLE_SIZE),
    axis.text  = element_text(size = AXIS_TEXT_SIZE)
  )

print(p_dist)

# Guardar PNG alta resolución (misma relación de aspecto 12 × 4.5)
dir.create(dirname(OUT_FIG), recursive = TRUE, showWarnings = FALSE)
ggsave(OUT_FIG, plot = p_dist, width = 12, height = 4.5, dpi = 600)

cat("Figura guardada en:\n", OUT_FIG, "\n")
cat("OK\n")


In [ ]:
library(readr)
library(dplyr)

OUT_DIR   <- "../results"
grad_dir  <- file.path(OUT_DIR, "asr_MCC_OU_gradient_sets")

# Distancias long: node, set, dist
OUT_DIST_CSV <- file.path(grad_dir, "gradient_distances_all_nodes.csv")

# Edades nodales del MCC
AGE_FILE <- file.path(OUT_DIR, "asr_MCC_OU_bootstrap", "node_uncertainty_with_age.csv")

dist_tbl <- read_csv(OUT_DIST_CSV, show_col_types = FALSE)
node_age <- read_csv(AGE_FILE, show_col_types = FALSE) %>% select(node, age_Mya)

dist_age <- dist_tbl %>%
  group_by(node) %>%
  summarise(
    dist_median = median(dist, na.rm = TRUE),
    dist_q025   = quantile(dist, 0.025, na.rm = TRUE),
    dist_q975   = quantile(dist, 0.975, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  inner_join(node_age, by = "node")

tab_S10 <- dist_age %>%
  mutate(
    ic_width = dist_q975 - dist_q025,
    age_band_id = ntile(desc(age_Mya), 5L)
  ) %>%
  group_by(age_band_id) %>%
  summarise(
    n_nodes          = n(),
    age_min          = min(age_Mya),
    age_max          = max(age_Mya),
    dist_median_min  = min(dist_median),
    dist_median_max  = max(dist_median),
    dist_median_mean = mean(dist_median),
    ic_width_mean    = mean(ic_width),
    .groups = "drop"
  ) %>%
  arrange(age_band_id) %>%
  mutate(across(where(is.numeric), ~ round(.x, 3))) %>%
  rename(
    `age band`         = age_band_id,
    `n nodes`          = n_nodes,
    `age min`          = age_min,
    `age max`          = age_max,
    `dist median min`  = dist_median_min,
    `dist median max`  = dist_median_max,
    `dist median mean` = dist_median_mean,
    `IC95 width mean`  = ic_width_mean
  )

OUT_S10 <- file.path(grad_dir, "Table_S10_sampling_uncertainty_5bands.csv")
write_csv(tab_S10, OUT_S10)

cat("Guardado S10:", OUT_S10, "\n")


In [ ]:
library(readr)
library(dplyr)

OUT_DIR   <- "../results"
grad_dir  <- file.path(OUT_DIR, "asr_MCC_OU_gradient_sets")

# Para saber cuántas dimensiones (D) y qué columnas usar
ASR_FILE  <- file.path(OUT_DIR, "asr_MCC_OU_nodes_high-level.csv")
asr_tbl   <- read_csv(ASR_FILE, show_col_types = FALSE)
trait_cols <- setdiff(colnames(asr_tbl), "node")
D <- length(trait_cols)

# Tips (mismo archivo)
TIPS_FILE <- "../traits/traits_high-level_aligned.csv"
tips_tbl  <- read_csv(TIPS_FILE, show_col_types = FALSE)

tips_vals  <- as.matrix(tips_tbl[, trait_cols])
tips_min   <- min(tips_vals, na.rm = TRUE)
tips_max   <- max(tips_vals, na.rm = TRUE)
tips_range <- tips_max - tips_min


uncert_per_dim <- dist_age %>%
  mutate(
    ic_width             = dist_q975 - dist_q025,
    step_median_per_dim  = dist_median / sqrt(D),
    step_icwidth_per_dim = ic_width    / sqrt(D)
  ) %>%
  summarise(
    mean_step_median_per_dim = mean(step_median_per_dim,  na.rm = TRUE),
    mean_step_ic_per_dim     = mean(step_icwidth_per_dim, na.rm = TRUE)
  )

mean_med_pct <- uncert_per_dim$mean_step_median_per_dim / tips_range * 100
mean_ic_pct  <- uncert_per_dim$mean_step_ic_per_dim     / tips_range * 100

tab_S11 <- tibble::tibble(
  tips_min              = tips_min,
  tips_max              = tips_max,
  tips_range            = tips_range,
  mean_step_med_per_dim = uncert_per_dim$mean_step_median_per_dim,
  mean_step_ic_per_dim  = uncert_per_dim$mean_step_ic_per_dim,
  step_med_pct_range    = mean_med_pct,
  step_ic_pct_range     = mean_ic_pct
) %>%
  rename(
    `tips min`        = tips_min,
    `tips max`        = tips_max,
    `tips range`      = tips_range,
    `median disp dim` = mean_step_med_per_dim,
    `IC95 disp dim`   = mean_step_ic_per_dim,
    `median disp %`   = step_med_pct_range,
    `IC95 disp %`     = step_ic_pct_range
  ) %>%
  mutate(across(where(is.numeric), ~ round(.x, 3)))

OUT_S11 <- file.path(grad_dir, "Table_S11_sampling_uncertainty_vs_tips_range.csv")
write_csv(tab_S11, OUT_S11)

cat("Guardado S11:", OUT_S11, "\n")
